In [1]:
import pandas as pd
import numpy as np

In [2]:
from araxia.network import MLP
from araxia.preprocessing import create_lagged_dataset
from araxia.train import normalize_features, normalize_targets, NormalizeFeatures
from araxia.utils import train_test_split

In [3]:
from atrax import Atrax as tx

In [4]:
df = pd.read_csv('sales.csv')

In [5]:
X_raw, y_raw = create_lagged_dataset(df, 'sale_date', 'sales', lag=5)

In [6]:
nf = NormalizeFeatures(X_raw)
X_norm = nf.normalize()

In [7]:
X_norm.shape

(479, 9)

In [13]:
#X_norm, X_mean, X_std = normalize_features(X_raw)

In [8]:
y_norm = normalize_targets(y_raw)
#

In [9]:
len(y_norm)

479

In [11]:
X_train, y_train, X_test, y_test = train_test_split(X_norm, y_norm, test_size=0.2)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [7]:
mlp = MLP(input_size=5, hidden_size=10, learning_rate=0.001)
mlp.train(X_norm, y_norm, epochs=300)

epoch: 0, | loss: 0.077127
epoch: 10, | loss: 0.008398
epoch: 20, | loss: 0.008265
epoch: 30, | loss: 0.008223
epoch: 40, | loss: 0.008211
epoch: 50, | loss: 0.008208
epoch: 60, | loss: 0.008208
epoch: 70, | loss: 0.008209
epoch: 80, | loss: 0.008209
epoch: 90, | loss: 0.008209
epoch: 100, | loss: 0.008210
epoch: 110, | loss: 0.008210
epoch: 120, | loss: 0.008210
epoch: 130, | loss: 0.008210
epoch: 140, | loss: 0.008210
epoch: 150, | loss: 0.008210
epoch: 160, | loss: 0.008210
epoch: 170, | loss: 0.008210
epoch: 180, | loss: 0.008210
epoch: 190, | loss: 0.008210
epoch: 200, | loss: 0.008210
epoch: 210, | loss: 0.008210
epoch: 220, | loss: 0.008210
epoch: 230, | loss: 0.008210
epoch: 240, | loss: 0.008210
epoch: 250, | loss: 0.008210
epoch: 260, | loss: 0.008210
epoch: 270, | loss: 0.008210
epoch: 280, | loss: 0.008210
epoch: 290, | loss: 0.008210


### forecasting

In [ ]:
future_steps = 10
X_future 

In [8]:
from datetime import timedelta
last_date = df['sale_date'].max()

In [9]:
future_dates = [last_date + timedelta(days=i) for i in range(1, 11)]

In [10]:
future_dates

[Timestamp('2025-06-22 00:00:00'),
 Timestamp('2025-06-23 00:00:00'),
 Timestamp('2025-06-24 00:00:00'),
 Timestamp('2025-06-25 00:00:00'),
 Timestamp('2025-06-26 00:00:00'),
 Timestamp('2025-06-27 00:00:00'),
 Timestamp('2025-06-28 00:00:00'),
 Timestamp('2025-06-29 00:00:00'),
 Timestamp('2025-06-30 00:00:00'),
 Timestamp('2025-07-01 00:00:00')]

### experiment

In [32]:
ds_test = tx.DataSet([
    {
        'date': '1/1/2025',
        'feature_1': 'a',
        'y': 0
    },
    {
        'date': '1/2/2025',
        'feature_1': 'b',
        'y': 1
    },
    {
        'date': '1/3/2025',
        'feature_1': 'c',
        'y': 2
    },
    {
        'date': '1/4/2025',
        'feature_1': 'd',
        'y': 3
    },
    {
        'date': '1/5/2025',
        'feature_1': 'e',
        'y': 4
    },
    {
        'date': '1/6/2025',
        'feature_1': 'f',
        'y': 5
    },
    {
        'date': '1/7/2025',
        'feature_1': 'g',
        'y': 6
    },
    {
        'date': '1/8/2025',
        'feature_1': 'h',
        'y': 7
    },
    {
        'date': '1/9/2025',
        'feature_1': 'i',
        'y': 8
    },
    {
        'date': '1/10/2025',
        'feature_1': 'j',
        'y': 9
    }
])

In [22]:
ds_test.shape()

(10, 3)

In [23]:
ds_test

date,feature_1,y
1/1/2025,a,0
1/2/2025,b,1
1/3/2025,c,2
1/4/2025,d,3
1/5/2025,e,4
1/6/2025,f,5
1/7/2025,g,6
1/8/2025,h,7
1/9/2025,i,8
1/10/2025,j,9


In [24]:
ds_test.iloc[:,:]

date,feature_1,y
1/1/2025,a,0
1/2/2025,b,1
1/3/2025,c,2
1/4/2025,d,3
1/5/2025,e,4
1/6/2025,f,5
1/7/2025,g,6
1/8/2025,h,7
1/9/2025,i,8
1/10/2025,j,9


In [25]:
ds_test.iloc[-1:, :]

date,feature_1,y
1/10/2025,j,9


In [28]:
X = ds_test.iloc[-1:, :]
print(X)

date, feature_1, y
1/10/2025, j, 9


In [35]:
ds_test.shape()

(10, 3)

In [59]:
X_train, y_train, X_test, y_test = train_test_split(ds_test, test_size=0.2)
print(f"X_train: {X_train}")
print(f"y_train: {y_train}")
print('--------')
print(f"X_test: {X_test}")
print(f"y_test: {y_test}")


X_train: date, feature_1
1/1/2025, a
1/2/2025, b
1/3/2025, c
1/4/2025, d
1/5/2025, e
1/6/2025, f
1/7/2025, g
1/8/2025, h
y_train: y
0
1
2
3
4
5
6
7
--------
X_test: date, feature_1
1/9/2025, i
1/10/2025, j
y_test: y
8
9
